In [1]:
import pickle

with open("bird.pkl", "rb") as f:
    c = pickle.load(f)

AttributeError: Can't get attribute 'Bird_Counter' on <module '__main__'>

In [2]:
class Bird_Counter:
    def __init__(self, variety=[], locations=[]):
        self.variety = variety
        self.locations = locations

    def get_birds(self):
        url = "https://api.ebird.org/v2/data/obs/US-UT/recent"
        payload = {}
        headers = {"X-eBirdApiToken": EBIRD_KEY}
        response = requests.request("GET", url, headers=headers, data=payload)
        self.variety = [x["comName"] for x in json.loads(response.text)]
        self.locations = [(x["lat"], x["lng"]) for x in json.loads(response.text)]

In [3]:
with open("bird.pkl", "rb") as f:
    c = pickle.load(f)

print(f"There were {len(c.variety)} birds.")
print(f"One of the birds was a {c.variety[0]}.")
print(f"One of the locations was {c.locations[0]}.")

There were 265 birds.
One of the birds was a Bald Eagle.
One of the locations was (40.4846248, -111.4994502).


Note: Our friend didn't even have `EBIRD_KEY`. This object was reconstructed from the binary pickle, not a call to `__init__` or any of the class methods.

## Dill

In [4]:
import dill

with open("dog.dill", "rb") as f:
    d = dill.load(f)

print(f"There were {len(d.variety)} dogs.")

There were 265 dogs.


Note: Our friend has no definition of `Dog_Counter`.

## msgpack

In [5]:
import msgpack

with open("msgpack_bird.bin", "rb") as f:
    result = msgpack.load(f)

e = Bird_Counter(**result)

print(f"There were {len(e.variety)} birds.")
print(f"One of the birds was a {e.variety[0]}.")
print(f"One of the locations was {e.locations[0]}.")

There were 265 birds.
One of the birds was a Bald Eagle.
One of the locations was [40.4846248, -111.4994502].


## Marshmallow

In [6]:
from marshmallow import Schema, fields, post_load


class CountSchema(Schema):
    variety = fields.List(fields.String)
    locations = fields.List(fields.Tuple((fields.Float, fields.Float)))

    @post_load
    def make_count(self, data, **kwargs):
        return Bird_Counter(**data)

In [7]:
import json

schema = CountSchema()
with open("marshmallow_bird.json", "r") as f:
    result = json.load(f)

f = schema.loads(result)
print(f"There were {len(f.variety)} birds.")
print(f"One of the birds was a {f.variety[0]}.")
print(f"One of the locations was {f.locations[0]}.")

There were 265 birds.
One of the birds was a Bald Eagle.
One of the locations was (40.4846248, -111.4994502).


Notice how in both msgpack and Marshmallow, we rely on having access to `Bird_Counter` and how we actually re-initialize a new object (in contrast to Dill).

[Do transporters kill you?](https://starships.com/teleporters-kill-you-the-science/)

Also note that our serialization format was really `json`. If you have things that are not json-able by default (scikit-learn classifiers, for example), you'll have to do write (de)serialization functions.

## New dev joins project

In [8]:
class FakeObject:
    def __init__(self, a=0, b=0, c=0):
        self.a = a
        self.b = b
        self.c = c

In [9]:
with open("fake_v1.pkl", "rb") as f:
    version2 = pickle.load(f)

version2.__dict__

{'a': 1, 'b': 1}

## Be careful, friend

In [10]:
with open("bad.pkl", "rb") as f:
    d = pickle.load(f)